# Test sign up and Sign in

In [ ]:
from lnhub_rest.orm._sbclient import connect_hub
from lnhub_rest._clean_ci import delete_ci_accounts
from lnhub_rest.core.account._signup_signin import sign_up_hub
from lnhub_rest.core.account._signup_signin import sign_in_hub
import jwt
import string, secrets
import os

# Ensure using production or staging environment to perform this test
# as we need autoconfirm supabase parameter to be off
# and testuser1 to exists
# to test sign up and sign in functions properly

# Update: (@lawrlee) The environment is now set in the test runner
# lamin_env = os.environ["LAMIN_ENV"]
# if lamin_env == "local":
#     os.environ["LAMIN_ENV"] = "staging"

In [ ]:
def base26(n_char: int):
    alphabet = string.ascii_lowercase
    return "".join(secrets.choice(alphabet) for i in range(n_char))

## Sign up

### Call `sign_up_hub`

In [ ]:
non_existing_account_email = f"lamin.ci.user.{base26(6)}@gmail.com"

In [ ]:
non_existing_account_password = sign_up_hub(non_existing_account_email)

In [ ]:
assert len(non_existing_account_password) == 40

### Call `sign_up_hub` on existing account

In [ ]:
existing_account_email = f"testuser1@lamin.ai"

In [ ]:
assert sign_up_hub(existing_account_email) == "user-exists"

## Sign in

### Call `sign_in_hub`

In [ ]:
existing_account_email = f"testuser1@lamin.ai"

In [ ]:
user_id, user_handle, user_name, access_token = sign_in_hub(
    existing_account_email, password="cEvcwMJFX4OwbsYVaMt2Os6GxxGgDUlBGILs2RyS"
)
session_payload = jwt.decode(
    access_token, algorithms="HS256", options={"verify_signature": False}
)

assert user_id == "DzTjkKse"
assert user_handle == "testuser1"
assert user_name == "Test User1"
assert session_payload["email"] == "testuser1@lamin.ai"

### Call `sign_in_hub` on a non completed signup account

In [ ]:
# Should be "complete-signup" but seems not possible to sign if sign up not completed
assert (
    sign_in_hub(non_existing_account_email, password=non_existing_account_password)
    == "could-not-login"
)

### Call `sign_in_hub` on a non existing account

In [ ]:
assert (
    sign_in_hub("non-existing-email@gmail.com", password="non-existing-password")
    == "could-not-login"
)

In [ ]:
# Switch back to local environment
if lamin_env == "local":
    os.environ["LAMIN_ENV"] = lamin_env